# Azure AI Search 테스트 노트북 - 창주스토어 상품 추천

창주스토어 상품 데이터를 기반으로 다양한 검색 옵션을 테스트합니다.

## 테스트할 검색 옵션
1. **Vector-only search** - Ada-002 또는 Large-3 임베딩만 사용
2. **Multi-vector search** - Ada-002와 Large-3 임베딩을 동시 활용  
3. **Hybrid search** - BM25 + Vector + Semantic reranker
4. **Query rewrite + Hybrid + Semantic** - 최고 성능 조합

## 검색 결과 출력 Helper

In [331]:
from azure.search.documents import SearchItemPaged

def print_results(results: SearchItemPaged[dict], query_text=""):
    """검색결과 포매팅"""
    
    print(f"검색어: '{query_text}'")
    print("=" * 100)
    
    # Semantic Answer가 있는 경우 먼저 출력
    semantic_answers = results.get_answers()
    if semantic_answers:
        print("Semantic Answer:")
        for answer in semantic_answers:
            if answer.highlights:
                print(f"   {answer.highlights}")
            else:
                print(f"   {answer.text}")
            print(f"   신뢰도: {answer.score:.3f}")
        print()
    
    result_count = 0
    for result in results:
        result_count += 1
        print(f"🛍️ 상품 {result_count}")
        print(f"   상품명: {result.get('title', 'N/A')}")
        print(f"   브랜드: {result.get('brand', 'N/A')}")
        print(f"   카테고리: {result.get('category', 'N/A')}")
        print(f"   가격: {result.get('normal_price', 0):,}원 ({result.get('price_range', 'N/A')})")
        print(f"   검색 점수: {result.get('@search.score', 0):.3f}")
        
        # Reranker 점수가 있는 경우 출력
        if result.get('@search.reranker_score'):
            print(f"   Reranker 점수: {result['@search.reranker_score']:.3f}")
        
        # Caption이 있는 경우 출력
        captions = result.get("@search.captions")
        if captions:
            caption = captions[0]
            if caption.highlights:
                print(f"   관련 내용: {caption.highlights}")
            else:
                print(f"   관련 내용: {caption.text}")
        
        print(f"   상품 정보: {result.get('main_text', 'N/A')[:100]}...")
        print("-" * 80)

# query rewrite 쿼리내용 출력. 테스트용으로만 확인하시고, 운영에서는 사용하시면 성능에 영향이 있습니다.
    try:
        print("\n Query Rewrite 디버그 정보 추출...")
    
        debug_info = None
        if hasattr(results, 'get_debug_info') and callable(getattr(results, 'get_debug_info')):
            try:
                debug_info = results.get_debug_info()
                print(f"   ✓ get_debug_info() 메서드로 디버그 정보 추출 성공")
            except Exception as e:
                print(f"   ✗ get_debug_info() 호출 실패: {e}")
        
        # Debug 정보 출력
        if debug_info:
            print(f"\n Query Rewrite 디버그 정보:")
            print(f"   • debug_info 타입: {type(debug_info)}")
            
            if hasattr(debug_info, 'query_rewrites'):
                query_rewrites = debug_info.query_rewrites
                print(f"   • query_rewrites 타입: {type(query_rewrites)}")
                
                if query_rewrites and hasattr(query_rewrites, 'text'):
                    text_rewrites = query_rewrites.text
                    print(f"   • text_rewrites 타입: {type(text_rewrites)}")
                    
                    if hasattr(text_rewrites, 'input_query'):
                        input_query = text_rewrites.input_query
                        print(f"   입력 쿼리: '{input_query}'")
                    if hasattr(text_rewrites, 'rewrites') and text_rewrites.rewrites:
                        rewrites = text_rewrites.rewrites
                        print(f"   {len(rewrites)}개의 리라이트된 쿼리:")
                        for i, rewrite in enumerate(rewrites, 1):
                            print(f"      {i}. '{rewrite}'")
                    else:
                        print("   리라이트된 쿼리: 없음")
                        # text_rewrites 객체의 속성들 확인
                        if hasattr(text_rewrites, '__dict__'):
                            attrs = list(text_rewrites.__dict__.keys())
                            print(f"   • text_rewrites 속성들: {attrs}")
                        else:
                            attrs = [attr for attr in dir(text_rewrites) if not attr.startswith('_')]
                            print(f"   • text_rewrites 메서드/속성들: {attrs}")
                else:
                    print("   query_rewrites 정보를 찾을 수 없습니다.")
            else:
                # Debug info 정보가 알 수 없는 패턴인 경우
                if hasattr(debug_info, '__dict__'):
                    attrs = list(debug_info.__dict__.keys())
                    print(f"   debug_info 속성: {attrs}")
                else:
                    attrs = [attr for attr in dir(debug_info) if not attr.startswith('_')]
                    print(f"   debug_info 메서드/속성: {attrs}")
            
            print("-" * 80)
        else:
            print("\n Debug 정보를 찾을 수 없습니다.")
            print("   Query Rewrite 기능이 활성화되지 않았거나 debug 파라미터가 누락되었을 수 있습니다.")
            
    except Exception as e:
        print(f"\n Debug 정보 추출 중 오류 발생: {str(e)}")
        import traceback
        print(f"   상세 오류: {traceback.format_exc()}")
        pass
    
    if result_count == 0:
        print("검색 결과가 없습니다.")
        
    print(f"\n총 {result_count}개 결과 표시\n")

## 환경 설정 및 클라이언트 초기화

In [332]:
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

# 필수 Azure Search 모델들만 import
try:
    from azure.search.documents.models import VectorizedQuery, QueryType, QueryAnswerType, QueryCaptionType
    print("Azure Search 모델들이 성공적으로 import되었습니다.")
except ImportError as e:
    print(f"Azure Search 모델 import 실패: {e}")

import os
load_dotenv(override=True)   

search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX")
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_key = os.getenv("AZURE_OPENAI_KEY")
ada002_deployment = os.getenv("AZURE_OPENAI_ADA002_EMBEDDING_DEPLOYMENT")
large3_deployment = os.getenv("AZURE_OPENAI_3_LARGE_EMBEDDING_DEPLOYMENT")

print(f"✓ Search endpoint: {search_endpoint}")
print(f"✓ Index name: {index_name}")
print(f"✓ Ada-002 deployment: {ada002_deployment}")
print(f"✓ Large-3 deployment: {large3_deployment}")

# 패키지 버전 확인
import azure.search.documents
print(f"azure-search-documents version: {azure.search.documents.__version__}")
print('beta 버전이 활성화되어 있어야 합니다. Query rewrite는 프리뷰 기능입니다~')

# 임베딩 클라이언트 초기화
embed_client = AzureOpenAI(
    azure_deployment=ada002_deployment,
    api_version="2023-05-15",
    azure_endpoint=openai_endpoint,
    api_key=openai_key,
)

embed3_client = AzureOpenAI(
    azure_deployment=large3_deployment,
    api_version="2023-05-15",
    azure_endpoint=openai_endpoint,
    api_key=openai_key,
)

search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(admin_key))

# 임베딩 생성 함수들
def get_embedding_ada002(query):
    return embed_client.embeddings.create(input=[query], model=ada002_deployment).data[0].embedding

def get_embedding_large3(query):
    return embed3_client.embeddings.create(input=[query], model=large3_deployment).data[0].embedding

print("✓ 임베딩 클라이언트 초기화 완료 (Ada-002, Large-3)")
print("✓ Azure Search 클라이언트 초기화 완료")


Azure Search 모델들이 성공적으로 import되었습니다.
✓ Search endpoint: https://hdfn-test-search.search.windows.net
✓ Index name: shopping-sample
✓ Ada-002 deployment: text-embedding-ada-002
✓ Large-3 deployment: text-embedding-3-large
azure-search-documents version: 11.6.0b12
beta 버전이 활성화되어 있어야 합니다. Query rewrite는 프리뷰 기능입니다~
✓ 임베딩 클라이언트 초기화 완료 (Ada-002, Large-3)
✓ Azure Search 클라이언트 초기화 완료


## 검색 옵션별 함수 정의
### 1. Vector-only Search

In [334]:
def vector_only_search(query_text, vector_field="main_text_vector", category_filter=None, top=5):
    """
    Vector-only 검색 (Ada-002 또는 Large-3)
    
    Args:
        query_text: 검색 쿼리
        vector_field: "main_text_vector" (Ada-002) 또는 "main_text_vector_3" (Large-3)  
        category_filter: 카테고리 필터 (예: "유아동", "패션의류")
        top: 반환할 결과 수
    """
    # 임베딩 생성
    if vector_field == "main_text_vector_3":
        embedding = get_embedding_large3(query_text)
        print(f"Large-3 Vector-only 검색")
    else:
        embedding = get_embedding_ada002(query_text)
        print(f"Ada-002 Vector-only 검색")
    
    vector_query = VectorizedQuery(
        vector=embedding, 
        k_nearest_neighbors=50, 
        fields=vector_field
    )
    
    # 필터 생성
    filter_condition = None
    if category_filter:
        filter_condition = f"category eq '{category_filter}'"
    
    results = search_client.search(
        search_text=None,  # 텍스트 검색 없이 벡터만 사용
        vector_queries=[vector_query],
        query_language="ko-kr",
        filter=filter_condition,
        top=top
    )
    
    return results

### 2. Multi-Vector Search

In [333]:
def multi_vector_search(query_text, category_filter=None, top=5):
    """
    Multi-vector 검색 (Ada-002 + Large-3 동시 사용)
    
    Args:
        query_text: 검색 쿼리
        category_filter: 카테고리 필터
        top: 반환할 결과 수
    """
    print(f"Multi-vector 검색 (Ada-002 + Large-3)")
    
    # 두 임베딩 생성
    embedding_ada = get_embedding_ada002(query_text)
    embedding_large3 = get_embedding_large3(query_text)
    
    vector_query1 = VectorizedQuery(
        vector=embedding_ada, 
        k_nearest_neighbors=25, 
        fields="main_text_vector"
    )
    
    vector_query2 = VectorizedQuery(
        vector=embedding_large3, 
        k_nearest_neighbors=25, 
        fields="main_text_vector_3"
    )
    
    # 필터 생성
    filter_condition = None
    if category_filter:
        filter_condition = f"category eq '{category_filter}'"
    
    results = search_client.search(
        search_text=None,
        vector_queries=[vector_query1, vector_query2],  # 두 벡터 동시 사용
        filter=filter_condition,
        query_language="ko-kr",
        top=top
    )
    
    return results

### 2.5. Vector Search with Semantic Reranker

순수 벡터 검색에 Semantic Reranker를 추가한 방식입니다.
- BM25 키워드 검색 없이 벡터 검색만 사용
- Semantic Reranker로 결과를 의미적으로 재순위화
- Hybrid 검색과 달리 키워드 매칭은 하지 않음

In [335]:
def vector_semantic_search(query_text, vector_field="main_text_vector_3", category_filter=None, top=5):
    """
    Vector 검색 + Semantic Reranker (BM25 없이 벡터 + 시맨틱만)
    
    Args:
        query_text: 검색 쿼리
        vector_field: "main_text_vector" (Ada-002) 또는 "main_text_vector_3" (Large-3)
        category_filter: 카테고리 필터
        top: 반환할 결과 수
    """
    # 임베딩 생성
    if vector_field == "main_text_vector_3":
        embedding = get_embedding_large3(query_text)
        print(f"Vector + Semantic 검색 (Large-3 + Semantic)")
    else:
        embedding = get_embedding_ada002(query_text)
        print(f"Vector + Semantic 검색 (Ada-002 + Semantic)")

    vector_query = VectorizedQuery(
        vector=embedding, 
        k_nearest_neighbors=50, 
        fields=vector_field
    )
    
    # 필터 생성
    filter_condition = None
    if category_filter:
        filter_condition = f"category eq '{category_filter}'"
    
    results = search_client.search(
        search_text=None,  # BM25 텍스트 검색 비활성화 (벡터만 사용)
        vector_queries=[vector_query],
        filter=filter_condition,
        query_type=QueryType.SEMANTIC,  # Semantic 활성화
        semantic_configuration_name="my-semantic-config",
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=top
    )
    
    return results

### 3. Hybrid Search with Semantic Reranker

In [339]:
def hybrid_semantic_search(query_text, vector_field="main_text_vector_3", category_filter=None, top=5):
    """
    Hybrid 검색 + Semantic Reranker (BM25 + Vector + Semantic)
    
    Args:
        query_text: 검색 쿼리
        vector_field: "main_text_vector" (Ada-002) 또는 "main_text_vector_3" (Large-3)
        category_filter: 카테고리 필터
        top: 반환할 결과 수
    """
    # 임베딩 생성
    if vector_field == "main_text_vector_3":
        embedding = get_embedding_large3(query_text)
        print(f"Hybrid + Semantic 검색 (BM25 + Large-3 + Semantic)")
    else:
        embedding = get_embedding_ada002(query_text)
        print(f"Hybrid + Semantic 검색 (BM25 + Ada-002 + Semantic)")
    
    vector_query = VectorizedQuery(
        vector=embedding, 
        k_nearest_neighbors=50, 
        fields=vector_field
    )
    
    # 필터 생성
    filter_condition = None
    if category_filter:
        filter_condition = f"category eq '{category_filter}'"
    
    results = search_client.search(
        search_text=query_text,  # BM25 텍스트 검색 활성화
        vector_queries=[vector_query],
        filter=filter_condition,
        query_type=QueryType.SEMANTIC,  # Semantic 활성화
        semantic_configuration_name="my-semantic-config",
        query_language="ko-kr",
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=top
    )
    
    return results

### 4. Query Rewrite + Hybrid + Semantic

In [337]:
def query_rewrite_hybrid_semantic_search(query_text, vector_field="main_text_vector_3", category_filter=None, top=5):
    """
    Query Rewrite + Hybrid + Semantic 검색 (최고 성능)
    
    Args:
        query_text: 검색 쿼리
        vector_field: "main_text_vector" (Ada-002) 또는 "main_text_vector_3" (Large-3)
        category_filter: 카테고리 필터
        top: 반환할 결과 수
    """
    # 임베딩 생성
    if vector_field == "main_text_vector_3":
        embedding = get_embedding_large3(query_text)
        print(f"Query Rewrite + Hybrid + Semantic 검색 (Large-3)")
    else:
        embedding = get_embedding_ada002(query_text)
        print(f"Query Rewrite + Hybrid + Semantic 검색 (Ada-002)")
    
    vector_query = VectorizedQuery(
        vector=embedding, 
        k_nearest_neighbors=50, 
        fields=vector_field
    )
    
    # 필터 생성
    filter_condition = None
    if category_filter:
        filter_condition = f"category eq '{category_filter}'"
    
    results = search_client.search(
        search_text=query_text,  # BM25 텍스트 검색
        vector_queries=[vector_query],
        filter=filter_condition,
        query_type=QueryType.SEMANTIC,  # Semantic 활성화
        semantic_configuration_name="my-semantic-config",
        query_language="ko-kr",
        query_rewrites="generative|count-10",  # Query Rewrite 활성화
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=top,
        debug="queryRewrites"  # 디버그 파라미터 단순화
        
    )
    
    return results

## 🧪 검색 테스트

### 검색 쿼리 입력
아래 셀에서 검색하고 싶은 상품을 입력합니다. (search_query 부분)

In [369]:
# 여기서 검색하고 싶은 상품을 입력하세요!
search_query = "신생아 장난감"

# 카테고리 필터 (옵션) - 빈 문자열이면 모든 카테고리 검색
category_filter = ""  # 예: "유아동", "패션의류", "이미용", "스포츠/레져" 등
# category_filter = ""  # 모든 카테고리 검색하려면 이렇게 설정

print(f"검색어: '{search_query}'")
if category_filter:
    print(f"카테고리 필터: '{category_filter}'")
else:
    print("카테고리 필터: 없음 (모든 카테고리 검색)")
print("=" * 50)

검색어: '신생아 장난감'
카테고리 필터: 없음 (모든 카테고리 검색)


In [370]:
# 옵션 1: 벡터 전용 검색 (Vector-only Search)
print("옵션 1: 벡터 전용 검색")
print("=" * 50)

results = vector_only_search(
    query_text=search_query,
    top=5,
    category_filter=category_filter if category_filter else None
)

print_results(results, search_query)

옵션 1: 벡터 전용 검색
Ada-002 Vector-only 검색
검색어: '신생아 장난감'
🛍️ 상품 1
   상품명: [풍천장어갤러리] 고창 풍천장어 선물세트 2kg (4∼5인분)
   브랜드: 고창풍천장어갤러리
   카테고리: 식품
   가격: 127,000원 (10만원대)
   검색 점수: 0.851
   상품 정보: [풍천장어갤러리] 고창 풍천장어 선물세트 2kg (4∼5인분) 고창풍천장어갤러리 식품 [창주스토어 목동점][풍천장어갤러리] 고창 풍천장어 선물세트 2kg (4∼5인분) #40A18...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: [10주년기념] 설봄X카카오프렌즈 춘식이 흑임자호박인절미 방배동서민갑부구름떡집
   브랜드: 설봄
   카테고리: 식품
   가격: 29,900원 (2만원대)
   검색 점수: 0.851
   상품 정보: [10주년기념] 설봄X카카오프렌즈 춘식이 흑임자호박인절미 방배동서민갑부구름떡집 설봄 식품 [창주스토어 본점][10주년기념] 설봄X카카오프렌즈 춘식이 흑임자호박인절미 방배동서민갑부구...
--------------------------------------------------------------------------------
🛍️ 상품 3
   상품명: [풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분)
   브랜드: 고창풍천장어갤러리
   카테고리: 식품
   가격: 189,000원 (10만원대)
   검색 점수: 0.850
   상품 정보: [풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분) 고창풍천장어갤러리 식품 [창주스토어 목동점][풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분) #40A18...
--------------------------------------------------------------------------------

In [364]:
# 옵션 2: 멀티 벡터 검색 (Multi-vector Search)
print("\n 옵션 2: 멀티 벡터 검색")
print("=" * 50)

results = multi_vector_search(
    query_text=search_query,
    top=5,
    category_filter=category_filter if category_filter else None
)

print_results(results, search_query)


 옵션 2: 멀티 벡터 검색
Multi-vector 검색 (Ada-002 + Large-3)
검색어: '아가 장난감'
🛍️ 상품 1
   상품명: 애니먼 강아지 고양이 습식사료 애니캔 플러스 신장·요로계 30g 6개
   브랜드: 애니먼
   카테고리: 문화/취미
   가격: 12,900원 (1만원대)
   검색 점수: 0.028
   상품 정보: 애니먼 강아지 고양이 습식사료 애니캔 플러스 신장·요로계 30g 6개 애니먼 문화/취미 [창주스토어 충청점]애니먼 강아지 고양이 습식사료 애니캔 플러스 신장·요로계 30g 6개 #...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: [풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분)
   브랜드: 고창풍천장어갤러리
   카테고리: 식품
   가격: 189,000원 (10만원대)
   검색 점수: 0.017
   상품 정보: [풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분) 고창풍천장어갤러리 식품 [창주스토어 목동점][풍천장어갤러리] 고창 풍천장어 선물세트 3kg (6∼7인분) #40A18...
--------------------------------------------------------------------------------
🛍️ 상품 3
   상품명: 5315-실바니안 그로서리 마켓
   브랜드: 토이트론
   카테고리: 유아동
   가격: 70,000원 (7만원대)
   검색 점수: 0.017
   상품 정보: 5315-실바니안 그로서리 마켓 토이트론 유아동 [창주스토어 판교점]5315-실바니안 그로서리 마켓 #60A2121954#토이트론#7만원대#70000...
--------------------------------------------------------------------------------
🛍️ 상품 4
   상품명: [풍천장어갤러리] 

In [365]:
# 옵션 2.5: 벡터 + 시맨틱 검색 (Vector + Semantic Search)
print("\n 옵션 2.5: 벡터 + 시맨틱 검색")
print("=" * 50)

results = vector_semantic_search(
    query_text=search_query,
    top=5,
    category_filter=category_filter if category_filter else None
)

print_results(results, search_query)


 옵션 2.5: 벡터 + 시맨틱 검색
Vector + Semantic 검색 (Large-3 + Semantic)
검색어: '아가 장난감'
🛍️ 상품 1
   상품명: 5315-실바니안 그로서리 마켓
   브랜드: 토이트론
   카테고리: 유아동
   가격: 70,000원 (7만원대)
   검색 점수: 0.634
   상품 정보: 5315-실바니안 그로서리 마켓 토이트론 유아동 [창주스토어 판교점]5315-실바니안 그로서리 마켓 #60A2121954#토이트론#7만원대#70000...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: 5807-실바니안 케이크 샵 스타터 세트
   브랜드: 토이트론
   카테고리: 유아동
   가격: 70,000원 (7만원대)
   검색 점수: 0.628
   상품 정보: 5807-실바니안 케이크 샵 스타터 세트 토이트론 유아동 [창주스토어 판교점]5807-실바니안 케이크 샵 스타터 세트 #60A2124151#토이트론#7만원대#70000...
--------------------------------------------------------------------------------
🛍️ 상품 3
   상품명: [압소바1] 곰고무연사목욕타올+손수건+인형세트 MIAZA-10911 [임신/출산선물]
   브랜드: 압소바
   카테고리: 유아동
   가격: 96,000원 (9만원대)
   검색 점수: 0.624
   상품 정보: [압소바1] 곰고무연사목욕타올+손수건+인형세트 MIAZA-10911 [임신/출산선물] 압소바 유아동 [창주스토어 미아점][압소바1] 곰고무연사목욕타올+손수건+인형세트 MIAZA-1...
--------------------------------------------------------------------------------
🛍️ 상품 4
   상품명: 피너츠 스누피 하우스 강아지 노

In [371]:
# 옵션 3: 하이브리드 + 시맨틱 검색 (Hybrid + Semantic Search)
print("\n 옵션 3: 하이브리드 + 시맨틱 검색")
print("=" * 50)

results = hybrid_semantic_search(
    query_text=search_query,
    top=5,
    category_filter=category_filter if category_filter else None
)

print_results(results, search_query)


 옵션 3: 하이브리드 + 시맨틱 검색
Hybrid + Semantic 검색 (BM25 + Large-3 + Semantic)
검색어: '신생아 장난감'
Semantic Answer:
   압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백<em>일 </em>선물) 압소바<em> 유아동</em> [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백일 선물) 2119554205#압소바#3만원대#30000. 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물).
   신뢰도: 0.902

🛍️ 상품 1
   상품명: 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물)
   브랜드: 압소바
   카테고리: 유아동
   가격: 39,000원 (3만원대)
   검색 점수: 0.032
   Reranker 점수: 2.424
   관련 내용: 압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백<em>일 </em>선물) 압소바<em> 유아동</em> [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백일 선물) 2119554205#압소바#3만원대#30000. 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물).
   상품 정보: 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물) 압소바 유아동 [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물) 21195...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: 애니먼 펫밀크 애니유 강아지 고양이 우유 180ml 10개
   브랜드: 애니먼
   카테고리: 문화/취미
   가격: 20,500원 (2만원대)
   검색 점수: 0.013
   Reranker 점수: 2.264
   관

In [372]:
# 옵션 4: 쿼리 리라이트 + 하이브리드 + 시맨틱 검색 (Query Rewrite + Hybrid + Semantic)
print("\n옵션 4: 쿼리 리라이트 + 하이브리드 + 시맨틱 검색")
print("=" * 50)

results = query_rewrite_hybrid_semantic_search(
    query_text=search_query,
    top=5,
    category_filter=category_filter if category_filter else None
)

print_results(results, search_query)


옵션 4: 쿼리 리라이트 + 하이브리드 + 시맨틱 검색
Query Rewrite + Hybrid + Semantic 검색 (Large-3)
검색어: '신생아 장난감'
Semantic Answer:
   압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백<em>일 </em>선물) 압소바<em> 유아동</em> [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백일 선물) 2119554205#압소바#3만원대#30000. 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물).
   신뢰도: 0.902

🛍️ 상품 1
   상품명: 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물)
   브랜드: 압소바
   카테고리: 유아동
   가격: 39,000원 (3만원대)
   검색 점수: 0.031
   Reranker 점수: 2.424
   관련 내용: 압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백<em>일 </em>선물) 압소바<em> 유아동</em> [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트<em>(신생아 </em>백일 선물) 2119554205#압소바#3만원대#30000. 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물).
   상품 정보: 압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물) 압소바 유아동 [창주스토어 목동점]압소바6 (ATA367P2M13) 레코딸랑이세트(신생아 백일 선물) 21195...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: 애니먼 펫밀크 애니유 강아지 고양이 우유 180ml 10개
   브랜드: 애니먼
   카테고리: 문화/취미
   가격: 20,500원 (2만원대)
   검색 점수: 0.013
   Reranker 점수: 2.2

## 🧪 추천 테스트 쿼리 - 검색 옵션별 특징 비교

각 검색 옵션의 장단점을 명확히 볼 수 있는 추천 쿼리들입니다:

### 💡 **Vector vs BM25 차이점 테스트**
```python
# 의미적 유사성 vs 키워드 정확성
search_query = "신생아 딸랑이" vs. "아기 딸랑이" (exact keyword vs. similar keyword)
```
- **Vector**: "유아 토이"도 "아기 장난감"과 유사한 결과 
- **Hybrid**: "신생아 딸랑이"에서 정확한 키워드 매칭으로 더 좋은 결과

### 🎯 **Ada-002 vs Large-3 임베딩 차이**
```python 
# 복합적 개념 검색
search_query = "집들이 선물"
search_query = "하우스워밍 선물"
search_query = "새집 이사 축하"
```
- **Ada-002**: 일반적인 의미 유사성
- **Large-3**: 더 정교한 의미 구분과 맥락 이해

### 🔥 **Semantic Reranker의 효과**
```python
# 브랜드/카테고리 혼재 쿼리
search_query = "앤더슨벨 가방" vs. "얜더슨밸 가방"
search_query = "라코스테 여성 티셔츠"
search_query = "노스페이스 자켓"
```
- **Vector Only**: 관련성 있지만 순위가 애매할 수 있음
- **Vector + Semantic**: 의미적으로 더 정확한 순위
- **Hybrid + Semantic**: BM25 + Vector + 의미적 재순위로 최적 결과

### 🚀 **Query Rewrite 테스트**
```python
# 불완전하거나 애매한 쿼리
search_query = "운동할 때 입는 옷"  # → "스포츠 의류", "운동복"으로 개선
search_query = "비올 때 옷"        
search_query = "출산 준비"        
```
- **일반 검색**: 애매한 쿼리로 부정확한 결과
- **Query Rewrite**: AI가 쿼리를 개선하여 훨씬 정확한 결과

### 🎁 **선물 추천 시나리오**
```python
# 선물 상황별 검색
search_query = "집들이 선물"           # 완벽한 매칭: 멜리멜로 디퓨저
search_query = "출산 선물"             # 다양한 딸랑이 세트들
search_query = "친구 생일 선물 3만원"   # 가격대 + 용도 조합
```

### 🔍 **브랜드 정확성 테스트** 
```python
# 브랜드명 정확도
search_query = "프리츠한센"      # 정확한 브랜드명
search_query = "Fritz Hansen"   # 영문 브랜드명
search_query = "덴마크 디자인"   # 브랜드 특성으로 검색
```

---

## 샘플 쿼리

### 🎯 **현재 추천 테스트 쿼리:**
```python
# 아래 중 하나를 선택해서 테스트해보세요!

# 1. 집들이 선물 (완벽한 매칭 테스트)
search_query = "집들이 선물"

# 2. 브랜드 + 카테고리 (정확성 테스트)  
search_query = "앤더슨벨 가방"

# 3. 애매한 쿼리 (Query Rewrite 효과)
search_query = "운동할 때 입는 옷"

# 4. 가격대 조합 (하이브리드 효과)
search_query = "3만원대 출산선물"
```

### 🎯 검색 옵션별 특징

1. **벡터 전용**: 의미적 유사성 기반 검색 (Ada-002 또는 Large-3)
2. **멀티 벡터**: 다중 임베딩 모델 활용 (Ada-002 + Large-3)
2.5. **벡터 + 시맨틱**: 벡터 검색 + 시맨틱 재순위화 (BM25 없음)
3. **하이브리드 + 시맨틱**: 키워드 + 벡터 + 시맨틱 리랭킹
4. **쿼리 리라이트**: AI가 쿼리를 개선하여 검색

In [235]:
# Query Rewrite 디버그 정보 테스트
print("🧪 Query Rewrite 디버그 정보 테스트")
print("=" * 50)

test_query = "라코스테 5만원대 선물"
print(f"테스트 쿼리: '{test_query}'")

results = query_rewrite_hybrid_semantic_search(
    query_text=test_query,
    top=3  # 결과를 3개로 제한
)

print_results(results, test_query)

🧪 Query Rewrite 디버그 정보 테스트
테스트 쿼리: '라코스테 5만원대 선물'
Query Rewrite + Hybrid + Semantic 검색 (Ada-002)
검색어: '라코스테 5만원대 선물'
Semantic Answer:
   사브르 비스트로 커트러리 빈티지 디너<em> 선물세트 </em>사브르 주방 [창주스토어 판교점]사브르 비스트로 커트러리 빈티지 디너<em> 선물세</em>트 #60A1995979#사브르<em>#5만원대#</em>50000. 사브르 비스트로 커트러리 빈티지 디너 선물세트.
   신뢰도: 0.977

🛍️ 상품 1
   상품명: 사브르 비스트로 커트러리 파스텔 빈티지 디너 선물세트
   브랜드: 사브르
   카테고리: 주방
   가격: 50,160원 (5만원대)
   검색 점수: 0.020
   Reranker 점수: 2.295
   관련 내용: 사브르 비스트로 커트러리 파스텔 빈티지 디너<em> 선물세</em>트 사브르 주방 [창주스토어 판교점]사브르 비스트로 커트러리 파스텔 빈티지 디너<em> 선물세</em>트 #60A1995971#사브르<em>#5만원대#</em>50000. 사브르 비스트로 커트러리 파스텔 빈티지 디너 선물세트.
   상품 정보: 사브르 비스트로 커트러리 파스텔 빈티지 디너 선물세트 사브르 주방 [창주스토어 판교점]사브르 비스트로 커트러리 파스텔 빈티지 디너 선물세트 #60A1995971#사브르#5만원대#5...
--------------------------------------------------------------------------------
🛍️ 상품 2
   상품명: 사브르 비스트로 커트러리 빈티지 디너 선물세트
   브랜드: 사브르
   카테고리: 주방
   가격: 50,160원 (5만원대)
   검색 점수: 0.012
   Reranker 점수: 2.271
   관련 내용: 사브르 비스트로 커트러리 빈티지 디너<em> 선물세트 </em>사브르 주방 [창주스토어 판교점]사브르 비스트로